USGS API Query for DEM 1/9 Arc resolution

In [3]:
import os
import arcpy
import requests
import logging
import errno
import datetime
from logging.handlers import RotatingFileHandler
from datetime import datetime

arcpy.env.overwriteOutput = True

# Define the API endpoint URL
url = "https://tnmaccess.nationalmap.gov/api/v1/products?datasets=National%20Elevation%20Dataset%20(NED)%201/9%20arc-second&max=999000"

# Define the query parameters

# Set the feature class or shapefile path

feature_class = r"C:\Project_Files\GIS\HMP+\Edenton\4.0 Reference Data\USGS\LiDAR_Processing\Working_Data\Other_Data\edenton_bb.shp"

# Set the file download Directory
download_directory = r"C:\Project_Files\GIS\HMP+\Edenton\4.0 Reference Data\USGS\LiDAR_Processing\Working_Data\Working_DEM\19arc"

# Define the date range (start date and end date)
start_date_str = "2010-01-01"
end_date_str = "2023-05-01"

# Convert start_date_str and end_date_str to datetime objects
start_date_datetime = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date_datetime = datetime.strptime(end_date_str, '%Y-%m-%d')

# Set log file 
SaveLogsTo = os.path.join(download_directory, "Download_Log")

#Setup logging - levels are DEBUG,INFO,WARNING,ERROR,CRITICAL
logging.basicConfig(level=logging.INFO)

''' ********************** SCRIPT CONFIGURATION END ********************** '''

#https://stackoverflow.com/questions/273192/how-can-i-create-a-directory-if-it-does-not-exist
def createFolder(folderPath):
    if not os.path.exists(folderPath):
        try:
            os.makedirs(folderPath)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

#Create specified folder if it does not exist already
createFolder(SaveLogsTo)

#Logging level specified in script configuration
logger = logging.getLogger(__name__)
logFileName = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
fileHandler = logging.handlers.RotatingFileHandler('{}/{}.log'.format(SaveLogsTo, logFileName), maxBytes=100000, backupCount=5)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(relativeCreated)d \n%(filename)s %(module)s %(funcName)s %(lineno)d \n%(message)s\n')
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)

offset = 0 # offset used to run with response query index offset - usually when download request times out or disconnects
items_to_download =[]
item_responses = 0

# Get the spatial extent of the feature class
desc = arcpy.Describe(feature_class)
spatial_extent = desc.extent

# Extract the extent properties
xmin = spatial_extent.XMin
ymin = spatial_extent.YMin
xmax = spatial_extent.XMax
ymax = spatial_extent.YMax

print(xmin, ymin, xmax, ymax)

extent_dd = spatial_extent.projectAs(arcpy.SpatialReference(4326))

# Extract the extent properties
xmin1 = extent_dd.XMin
ymin1 = extent_dd.YMin
xmax1 = extent_dd.XMax
ymax1 = extent_dd.YMax

print(xmin1, ymin1, xmax1, ymax1)

params = {
    "bbox": f"{xmin1},{ymin1},{xmax1},{ymax1}"
}

logger.info("Initiating API Query:: {0}".format(str(datetime.now())))
while True:
    # Create the API URL with the current offset
    url = f"{url}&offset={offset}"

    # Send the API request and retrieve the response
    response = requests.get(url, params=params)


    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()["items"]

        # If there are no items in the response, we have retrieved all items
        if len(data) == 0:
            break

        for index, item in enumerate(data):
            item_responses += 1
            pub_date = datetime.strptime(item["publicationDate"], "%Y-%m-%d")
            download_url = item["downloadURL"]
            if start_date_datetime <= pub_date <= end_date_datetime:
                if download_url not in [item["download_url"] for item in items_to_download]: # only unique instances of the download url will be added to the dictionary from the query results
                    items_to_download.append({
                        "publication_date": item["publicationDate"],
                        "download_url": item["downloadURL"],
                        "title": item["title"],
                        "response_index": index + 1})
            
        # Increment the offset for the next page of items
        offset += 999

logger.info(f"Total items returned from API Query: {item_responses}")
logger.info(f"Total items added to download list: {len(items_to_download)}")

# Iterate over each item in the items to download list and download them
logger.info("Initiating TIFF file download from USGS:: {0}".format(datetime.now()))
for index, item in enumerate(items_to_download):
    publication_date = item["publication_date"]
    download_url = item["download_url"]
    title = item["title"]
    response = item["response_index"]

    # Specify the local file path to save the downloaded file
    rename = title.replace(" ","_") # replace spaces in name with underscores
    renamex = rename.replace("/","_")
    save_path = os.path.join(download_directory, renamex + ".tif")

        # Download the file and save it locally
    with open(save_path, "wb") as file:
        file.write(requests.get(download_url).content)
        logger.info(f"saved item : : \nTitle: {title}\n{index + 1} of {len(items_to_download)} items to download\nRespone Query Index : {response}")
        arcpy.SetProgressorPosition()

INFO:__main__:Initiating API Query:: 2023-12-04 12:49:03.358439


392788.4844000004 4704571.299900001 498367.7719000004 4724822.046900001
-88.30832095063823 42.48607514947249 -87.01986291640718 42.67587109009426


INFO:__main__:Total items returned from API Query: 4
INFO:__main__:Total items added to download list: 4
INFO:__main__:Initiating TIFF file download from USGS:: 2023-12-04 12:49:05.195333
INFO:__main__:saved item : : 
Title: USGS NED ned19_n42x50_w088x00_il_lakeco_2007 1/9 arc-second 2011 15 x 15 minute IMG
1 of 4 items to download
Respone Query Index : 1
INFO:__main__:saved item : : 
Title: USGS NED ned19_n42x50_w088x25_il_lakeco_2007 1/9 arc-second 2011 15 x 15 minute IMG
2 of 4 items to download
Respone Query Index : 2
INFO:__main__:saved item : : 
Title: USGS NED ned19_n42x50_w088x25_il_mchenryco_2008 1/9 arc-second 2009 15 x 15 minute IMG
3 of 4 items to download
Respone Query Index : 3
INFO:__main__:saved item : : 
Title: USGS NED ned19_n42x50_w088x50_il_mchenryco_2008 1/9 arc-second 2009 15 x 15 minute IMG
4 of 4 items to download
Respone Query Index : 4


USGS API Query for DEM 1/3 Arc resolution

In [15]:
import os
import arcpy
import requests
import logging
import errno
import datetime
from logging.handlers import RotatingFileHandler
from datetime import datetime

arcpy.env.overwriteOutput = True

# Define the API endpoint URL
url = "https://tnmaccess.nationalmap.gov/api/v1/products?datasets=National%20Elevation%20Dataset%20(NED)%201/3%20arc-second&max=999000"

# Define the query parameters

# Set the feature class or shapefile path

feature_class = r"C:\Project_Files\GIS\FFRMS+\Counties+\OH_39027+\Working_Data\Other_Data\Clinton_17.shp"

# Set the file download Directory
download_directory = r"C:\Project_Files\GIS\FFRMS+\Counties+\OH_39027+\Working_Data\Working_DEM\01m"

# Define the date range (start date and end date)
start_date_str = "2020-02-01"
end_date_str = "2020-04-01"

# Convert start_date_str and end_date_str to datetime objects
start_date_datetime = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date_datetime = datetime.strptime(end_date_str, '%Y-%m-%d')

# Set log file 
SaveLogsTo = os.path.join(download_directory, "Download_Log")

#Setup logging - levels are DEBUG,INFO,WARNING,ERROR,CRITICAL
logging.basicConfig(level=logging.INFO)

''' ********************** SCRIPT CONFIGURATION END ********************** '''

#https://stackoverflow.com/questions/273192/how-can-i-create-a-directory-if-it-does-not-exist
def createFolder(folderPath):
    if not os.path.exists(folderPath):
        try:
            os.makedirs(folderPath)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

#Create specified folder if it does not exist already
createFolder(SaveLogsTo)

#Logging level specified in script configuration
logger = logging.getLogger(__name__)
logFileName = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
fileHandler = logging.handlers.RotatingFileHandler('{}/{}.log'.format(SaveLogsTo, logFileName), maxBytes=100000, backupCount=5)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(relativeCreated)d \n%(filename)s %(module)s %(funcName)s %(lineno)d \n%(message)s\n')
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)

offset = 0 # offset used to run with response query index offset - usually when download request times out or disconnects
items_to_download =[]
item_responses = 0

# Get the spatial extent of the feature class
desc = arcpy.Describe(feature_class)
spatial_extent = desc.extent

# Extract the extent properties
xmin = spatial_extent.XMin
ymin = spatial_extent.YMin
xmax = spatial_extent.XMax
ymax = spatial_extent.YMax

print(xmin, ymin, xmax, ymax)

extent_dd = spatial_extent.projectAs(arcpy.SpatialReference(4326))

# Extract the extent properties
xmin1 = extent_dd.XMin
ymin1 = extent_dd.YMin
xmax1 = extent_dd.XMax
ymax1 = extent_dd.YMax

print(xmin1, ymin1, xmax1, ymax1)

params = {
    "bbox": f"{xmin1},{ymin1},{xmax1},{ymax1}"
}

logger.info("Initiating API Query:: {0}".format(str(datetime.now())))
while True:
    # Create the API URL with the current offset
    url = f"{url}&offset={offset}"

    # Send the API request and retrieve the response
    response = requests.get(url, params=params)


    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()["items"]

        # If there are no items in the response, we have retrieved all items
        if len(data) == 0:
            break

        for index, item in enumerate(data):
            item_responses += 1
            pub_date = datetime.strptime(item["publicationDate"], "%Y-%m-%d")
            download_url = item["downloadURL"]
            if start_date_datetime <= pub_date <= end_date_datetime:
                if download_url not in [item["download_url"] for item in items_to_download]: # only unique instances of the download url will be added to the dictionary from the query results
                    items_to_download.append({
                        "publication_date": item["publicationDate"],
                        "download_url": item["downloadURL"],
                        "title": item["title"],
                        "response_index": index + 1})
            
        # Increment the offset for the next page of items
        offset += 999

logger.info(f"Total items returned from API Query: {item_responses}")
logger.info(f"Total items added to download list: {len(items_to_download)}")

# Iterate over each item in the items to download list and download them
logger.info("Initiating TIFF file download from USGS:: {0}".format(datetime.now()))
for index, item in enumerate(items_to_download):
    publication_date = item["publication_date"]
    download_url = item["download_url"]
    title = item["title"]
    response = item["response_index"]

    # Specify the local file path to save the downloaded file
    rename = title.replace(" ","_") # replace spaces in name with underscores
    renamex = rename.replace("/","_")
    save_path = os.path.join(download_directory, renamex + ".tif")

        # Download the file and save it locally
    with open(save_path, "wb") as file:
        file.write(requests.get(download_url).content)
        logger.info(f"saved item : : \nTitle: {title}\n{index + 1} of {len(items_to_download)} items to download\nRespone Query Index : {response}")
        arcpy.SetProgressorPosition()


INFO:__main__:Initiating API Query:: 2023-11-01 14:03:05.750694


508800.55179999955 5054368.9169 575494.5899 5111996.793400001
-92.88706968404222 45.63876090354563 -92.0222473421357 46.16149077419841


INFO:__main__:Total items returned from API Query: 6
INFO:__main__:Total items added to download list: 6
INFO:__main__:Initiating TIFF file download from USGS:: 2023-11-01 14:03:07.326637


USGS 1/3 Arc Second n46w093 20211124
USGS 1/3 Arc Second n46w093 20230210
USGS 1/3 Arc Second n47w093 20211124
USGS 1/3 Arc Second n47w093 20221115
USGS 1/3 arc-second n46w093 1 x 1 degree
USGS 1/3 arc-second n47w093 1 x 1 degree


INFO:__main__:saved item : : 
Title: USGS 1/3 Arc Second n46w093 20211124
1 of 6 items to download
Respone Query Index : 1
INFO:__main__:saved item : : 
Title: USGS 1/3 Arc Second n46w093 20230210
2 of 6 items to download
Respone Query Index : 2
INFO:__main__:saved item : : 
Title: USGS 1/3 Arc Second n47w093 20211124
3 of 6 items to download
Respone Query Index : 3
INFO:__main__:saved item : : 
Title: USGS 1/3 Arc Second n47w093 20221115
4 of 6 items to download
Respone Query Index : 4
INFO:__main__:saved item : : 
Title: USGS 1/3 arc-second n46w093 1 x 1 degree
5 of 6 items to download
Respone Query Index : 5
INFO:__main__:saved item : : 
Title: USGS 1/3 arc-second n47w093 1 x 1 degree
6 of 6 items to download
Respone Query Index : 6


USGS API Query for 1m DEM

In [3]:
import os
import arcpy
import requests
import logging
import errno
import datetime
from logging.handlers import RotatingFileHandler
from datetime import datetime

arcpy.env.overwriteOutput = True

# Define the API endpoint URL
url = "https://tnmaccess.nationalmap.gov/api/v1/products?datasets=Digital%20Elevation%20Model%20(DEM)%201%20meter&max=999000"

# Define the query parameters

# Set the feature class or shapefile path

feature_class = r"C:\Project_Files\GIS\HMP+\Edenton\4.0 Reference Data\USGS\LiDAR_Processing\Working_Data\Other_Data\edenton_bb.shp"

# Set the file download Directory
download_directory = r"C:\Project_Files\GIS\HMP+\Edenton\4.0 Reference Data\USGS\LiDAR_Processing\Working_Data\Working_DEM\01m"

# Define the date range (start date and end date)
start_date_str = "2010-01-01"
end_date_str = "2023-05-01"

# Convert start_date_str and end_date_str to datetime objects
start_date_datetime = datetime.strptime(start_date_str, '%Y-%m-%d')
end_date_datetime = datetime.strptime(end_date_str, '%Y-%m-%d')

# Set log file 
SaveLogsTo = os.path.join(download_directory, "Download_Log")

#Setup logging - levels are DEBUG,INFO,WARNING,ERROR,CRITICAL
logging.basicConfig(level=logging.INFO)

''' ********************** SCRIPT CONFIGURATION END ********************** '''

#https://stackoverflow.com/questions/273192/how-can-i-create-a-directory-if-it-does-not-exist
def createFolder(folderPath):
    if not os.path.exists(folderPath):
        try:
            os.makedirs(folderPath)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

#Create specified folder if it does not exist already
createFolder(SaveLogsTo)

#Logging level specified in script configuration
logger = logging.getLogger(__name__)
logFileName = datetime.now().strftime('%Y-%m-%d %H-%M-%S')
fileHandler = logging.handlers.RotatingFileHandler('{}/{}.log'.format(SaveLogsTo, logFileName), maxBytes=100000, backupCount=5)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(relativeCreated)d \n%(filename)s %(module)s %(funcName)s %(lineno)d \n%(message)s\n')
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)

offset = 0 # offset used to run with response query index offset - usually when download request times out or disconnects
items_to_download =[]
item_responses = 0

# Get the spatial extent of the feature class
desc = arcpy.Describe(feature_class)
spatial_extent = desc.extent

# Extract the extent properties
xmin = spatial_extent.XMin
ymin = spatial_extent.YMin
xmax = spatial_extent.XMax
ymax = spatial_extent.YMax

print(xmin, ymin, xmax, ymax)

extent_dd = spatial_extent.projectAs(arcpy.SpatialReference(4326))

# Extract the extent properties
xmin1 = extent_dd.XMin
ymin1 = extent_dd.YMin
xmax1 = extent_dd.XMax
ymax1 = extent_dd.YMax

print(xmin1, ymin1, xmax1, ymax1)

params = {
    "bbox": f"{xmin1},{ymin1},{xmax1},{ymax1}"
}

logger.info("Initiating API Query:: {0}".format(str(datetime.now())))
while True:
    # Create the API URL with the current offset
    url = f"{url}&offset={offset}"

    # Send the API request and retrieve the response
    response = requests.get(url, params=params)


    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the response JSON
        data = response.json()["items"]

        # If there are no items in the response, we have retrieved all items
        if len(data) == 0:
            break

        for index, item in enumerate(data):
            item_responses += 1
            pub_date = datetime.strptime(item["publicationDate"], "%Y-%m-%d")
            download_url = item["downloadURL"]
            if start_date_datetime <= pub_date <= end_date_datetime:
                if download_url not in [item["download_url"] for item in items_to_download]: # only unique instances of the download url will be added to the dictionary from the query results
                    items_to_download.append({
                        "publication_date": item["publicationDate"],
                        "download_url": item["downloadURL"],
                        "title": item["title"],
                        "response_index": index + 1})
            
        # Increment the offset for the next page of items
        offset += 999

logger.info(f"Total items returned from API Query: {item_responses}")
logger.info(f"Total items added to download list: {len(items_to_download)}")

# Iterate over each item in the items to download list and download them
logger.info("Initiating TIFF file download from USGS:: {0}".format(datetime.now()))
for index, item in enumerate(items_to_download):
    publication_date = item["publication_date"]
    download_url = item["download_url"]
    title = item["title"]
    response = item["response_index"]

    # Specify the local file path to save the downloaded file
    rename = title.replace(" ","_") # replace spaces in name with underscores
    renamex = rename.replace("/","_")
    save_path = os.path.join(download_directory, renamex + ".tif")

        # Download the file and save it locally
    with open(save_path, "wb") as file:
        file.write(requests.get(download_url).content)
        logger.info(f"saved item : : \nTitle: {title}\n{index + 1} of {len(items_to_download)} items to download\nRespone Query Index : {response}")
        arcpy.SetProgressorPosition()

INFO:__main__:Initiating API Query:: 2023-12-12 13:16:31.661130


2676837.2796126306 825761.5556894839 2728988.418748464 871059.5494109006
-76.7117083647688 35.993993796206844 -76.53165547368441 36.12182421674153


INFO:__main__:Total items returned from API Query: 0
INFO:__main__:Total items added to download list: 0
INFO:__main__:Initiating TIFF file download from USGS:: 2023-12-12 13:16:33.373237
